In [1]:
import pandas as pd
import numpy as np

# Combine all dependent variables

In [2]:
# crosswalk - standardize with ISO3 at the end
crosswalk = pd.read_excel("../../data/country_names.xlsx")
# coal variables
coal_var = pd.read_csv("../_data_process/_all_temporal_coal_indicators.csv")
# gas variables
gas_var = pd.read_csv("../_data_process/_all_temporal_gas_indicators.csv")
# generation variables
generation_var = pd.read_csv("../growth_rate/_generation_results_data.csv")

In [3]:
coal_var_rename = pd.merge(coal_var,crosswalk[["Country Code","GEM_Name"]],left_on="Country",right_on="GEM_Name",how="left")
# some countries are not in the crosswalk databse
dropped_countries = coal_var_rename[coal_var_rename["Country Code"].isna()]["Country"].unique()
coal_var_rename = coal_var_rename.drop(columns=["GEM_Name","Country"])
coal_var_rename = coal_var_rename.rename(columns={"Year":"year"})
coal_var_rename = coal_var_rename[coal_var_rename["Country Code"].isna()==False]
coal_var_rename = coal_var_rename.drop_duplicates()
coal_var_rename

,year,Coal_Current_MW,Coal_AnnualMtCO2,Coal_start_year,Coal_Peak_MW,Coal_Ret_MW,Coal_EarlyRet_MW,Coal_Tran_MW,Coal_duration_year,Coal_Share_Ret,Coal_Share_EarlyRet,Coal_Share_Tran,Coal_Share_PhasedOut,Coal_logCurrent_MW,Country Code
0,1927,36.0,0.2,1927.0,36.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,3.583519,DEU
1,1928,36.0,0.2,1927.0,36.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000,0.0,3.583519,DEU
2,1929,36.0,0.2,1927.0,36.0,0.0,0.0,0.0,2.0,0.000000,0.0,0.000000,0.0,3.583519,DEU
3,1930,36.0,0.2,1927.0,36.0,0.0,0.0,0.0,3.0,0.000000,0.0,0.000000,0.0,3.583519,DEU
4,1931,36.0,0.2,1927.0,36.0,0.0,0.0,0.0,4.0,0.000000,0.0,0.000000,0.0,3.583519,DEU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3966,2022,0.0,0.0,1949.0,2865.0,2188.0,0.0,677.0,73.0,76.369983,0.0,23.630017,100.0,0.000000,BEL
3967,2022,0.0,0.0,1999.0,135.0,135.0,135.0,0.0,23.0,100.000000,100.0,0.000000,100.0,0.000000,PER
3968,2022,0.0,0.0,1964.0,2028.0,1296.0,0.0,732.0,58.0,63.905325,0.0,36.094675,100.0,0.000000,PRT
3969,2022,0.0,0.0,1963.0,291.0,291.0,0.0,0.0,59.0,100.000000,0.0,0.000000,100.0,0.000000,SWE


In [10]:
coal_gas = pd.merge(coal_var_rename,gas_var,on = ["Country Code","year"],how="outer")
coal_gas = coal_gas[coal_gas["year"]!=2023] # remove because coal and gas data are from early 2023
coal_gas = coal_gas.fillna(0) # assume those without corresponding data to be 0 

coal_gas = pd.merge(coal_gas,generation_var,left_on = ["Country Code","year"],right_on = ["Code","Year"],how="outer")
# remove those without capacity data 
coal_gas = coal_gas[coal_gas["Country Code"].isna()==False]
# remove those without generation data 
coal_gas = coal_gas[coal_gas["Code"].isna()==False]
coal_gas = coal_gas.drop(columns=["Code","Year","Coal_start_year","Gas_start_year"])
coal_gas = coal_gas.set_index(["Country Code","year"])
coal_gas = coal_gas.sort_values(by="year")
coal_gas["Share_VRE_Gen"] = coal_gas[['Share_Solar_Gen', 'Share_Wind_Gen']].sum(axis=1)
coal_gas.to_csv("_all_temporal_coalgas_var.csv")
coal_gas

,,Coal_Current_MW,Coal_AnnualMtCO2,Coal_Peak_MW,Coal_Ret_MW,Coal_EarlyRet_MW,Coal_Tran_MW,Coal_duration_year,Coal_Share_Ret,Coal_Share_EarlyRet,Coal_Share_Tran,...,Share_Gas_Gen,Share_Coal_Gen,Coal_Gen_Decline_Perc,Gas_Gen_Decline_Perc,rate_coal_gen_5y,rate_coal_share_5y,rate_coal_cap_5y,rate_gas_gen_5y,rate_gas_share_5y,Share_VRE_Gen
Country Code,year,,,,,,,,,,,,,,,,,,,,,
ARG,1990.0,375.0,2.0,375.0,0.0,0.0,0.0,7.0,0.000000,0.000000,0.000000,...,0.389172,0.012940,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
SRB,1990.0,4087.0,23.5,4087.0,0.0,0.0,0.0,34.0,0.000000,0.000000,0.000000,...,0.033872,0.725686,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
SVK,1990.0,1285.0,7.5,1285.0,0.0,0.0,0.0,33.0,0.000000,0.000000,0.000000,...,0.071401,0.294233,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
SVN,1990.0,1004.0,6.1,1004.0,0.0,0.0,0.0,34.0,0.000000,0.000000,0.000000,...,0.000000,0.312701,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
ZAF,1990.0,32239.3,170.7,32239.3,0.0,0.0,0.0,38.0,0.000000,0.000000,0.000000,...,0.000000,0.932427,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DNK,2022.0,1560.0,8.0,5003.0,3208.0,371.0,235.0,60.0,64.121527,7.415551,4.697182,...,0.029924,0.105171,90.874716,0.000000,NaN,NaN,NaN,NaN,NaN,0.608077
CZE,2022.0,7499.6,43.9,8951.1,1786.5,0.0,1210.0,71.0,19.958441,0.000000,13.517892,...,0.078134,0.434379,31.269753,0.000000,NaN,NaN,NaN,NaN,NaN,0.037128
HRV,2022.0,335.0,1.9,335.0,0.0,0.0,0.0,53.0,0.000000,0.000000,0.000000,...,0.257082,0.109065,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0.171388
